In [2]:
import os
import re
import json
import numpy as np
from PIL import Image
from tqdm import tqdm
import torch
import random
import pybullet as p
import pybullet_data
import trimesh
from collections import defaultdict, Counter

# Create Object Files

In [1]:
from sim.utils import create_obj
import shutil

def clear_files():
    for object_name in os.listdir("objects"):
        if object_name in ['sphere']:
            continue

        object_path = os.path.join("objects", object_name)
        for object_id in os.listdir(object_path):
            object_id_path = os.path.join(object_path, object_id)

            for f_name in os.listdir(object_id_path):
                f_path = os.path.join(object_id_path, f_name)

                # Remove 'assets' directory if it exists
                if f_name == "assets" and os.path.isdir(f_path):
                    try:
                        shutil.rmtree(f_path)
                        print(f"Removed directory: {f_path}")
                    except Exception as e:
                        print(f"Error removing directory {f_path}: {e}")
                    continue  # Skip further checks for this item

                # Check if it is a file (not a directory)
                if os.path.isfile(f_path):
                    # Keep only files ending with .glb or .blend
                    if not (f_name.endswith('.glb') or f_name.endswith('.blend')):
                        try:
                            os.remove(f_path)
                            print(f"Removed file: {f_path}")
                        except Exception as e:
                            print(f"Error deleting {f_path}: {e}")

object_specs = [
    ('table', 0.5, False),
    ('knife', 0.04, False),
    ('spoon', 0.04, False),
    ('fork', 0.04, False),
    ('apple', 0.2, False),
    ('banana', 0.2, False),
    ('pear', 0.2, False),
    ('mug', 0.5, True),
    ('paper-cup', 0.3, True),
    ('bowl', 0.4, True),
    ('box', 0.3, True),
    ('basket', 0.3, True),
    ('pan', 0.7, True)
]

# clear_files()

# for name, mass, convex in object_specs:
#     create_obj(name, mass, convex)


# Find Object Sizes

In [3]:
from core.env.scene_manager import OBJECTS

def get_object_diameter(obj_name, body_type, object_dir="sim\objects"):
    mesh_path = f"{object_dir}/{obj_name}/{obj_name}_{body_type}/{obj_name}_{body_type}.obj"
    mesh = trimesh.load_mesh(mesh_path)
    extents = mesh.bounding_box.extents
    diameter = np.max(extents[:2])  # XY-plane diameter
    if obj_name in ['basket', 'box']:
        diameter = (diameter + np.linalg.norm(extents[:2])) / 2
    return diameter

def compute_max_diameters(object_dir="sim\objects"):
    p.connect(p.DIRECT)
    max_diameters = {}

    object_names = [d for d in os.listdir(object_dir) if os.path.isdir(os.path.join(object_dir, d))]

    for name in object_names:
        try:
            body_types = [
                int(folder.split("_")[-1])
                for folder in os.listdir(f"{object_dir}/{name}")
                if folder.startswith(f"{name}_")
            ]
        except Exception as e:
            print(f"Skipping {name}: {e}")
            continue

        diameters = []
        for body_type in sorted(body_types):
            try:
                diameter = get_object_diameter(name, body_type, object_dir)
                diameters.append(diameter)
            except Exception as e:
                print(f"Failed to process {name} bodyType {body_type}: {e}")
        if diameters:
            max_diameters[name] = np.round(max(diameters), 2)

    p.disconnect()
    return max_diameters

# Update sizes in OBJECTS
max_diameters = compute_max_diameters()
for obj in OBJECTS.values():
	name = obj["name"]
	if name in max_diameters:
		size = np.round(max_diameters[name] * 100)
		if size % 2 == 0:
			size += 1
		obj["size"] = (size, size)

object_sizes = {k: v["size"] for k, v in OBJECTS.items()}
object_sizes

Skipping table: invalid literal for int() with base 10: '1.1'


{0: (19.0, 19.0),
 1: (19.0, 19.0),
 2: (19.0, 19.0),
 3: (9.0, 9.0),
 4: (7.0, 7.0),
 5: (17.0, 17.0),
 6: (9.0, 9.0),
 7: (13.0, 13.0),
 8: (13.0, 13.0),
 9: (25.0, 25.0),
 10: (23.0, 23.0),
 11: (27.0, 27.0)}

# Create Meta File

In [4]:
from core.env.scene_manager import SceneManager
from core.env.scene_utils import create_scene_meta, save_scene_meta, get_next_scene_id

def save_rearrangement_meta(dataset_dir, grid_size, initial_x, target_x, scene_id):
	subfolder_name = f'rearrangement_{scene_id:05d}'
	subfolder_path = os.path.join(dataset_dir, subfolder_name)
	
	# Use shared utility functions
	scene_meta = create_scene_meta(initial_x, target_x, scene_id, grid_size)
	meta_path = os.path.join(subfolder_path, 'meta.json')
	save_scene_meta(scene_meta, meta_path, verbose=True)

# --- Main execution ---
num_cases = 10
grid_size = (100, 100)
dataset_dir = 'shit'

# Get starting scene ID
scene_id = get_next_scene_id(dataset_dir, prefix='rearrangement')

for num_objects in [7, 8, 9, 10]:
	print(f'Generating {num_cases} scenes with {num_objects} objects...')
	
	# Create environment with custom object sizes
	env = SceneManager(
		num_objects=num_objects,
		grid_size=grid_size,
		verbose=0
	)
	
	# Override with your computed object sizes
	env.object_sizes = object_sizes
	
	for _ in range(num_cases):
		# Generate scene using SceneManager
		env.reset(use_stack=True, use_sides=False)  # stack_prob=1 equivalent
		
		initial_x = env.initial_x.clone()
		target_x = env.target_x.clone()
		
		save_rearrangement_meta(dataset_dir, grid_size, initial_x, target_x, scene_id)
		scene_id += 1


Generating 10 scenes with 7 objects...
Saved shit\rearrangement_00000\meta.json
Saved shit\rearrangement_00001\meta.json
Saved shit\rearrangement_00002\meta.json
Saved shit\rearrangement_00003\meta.json
Saved shit\rearrangement_00004\meta.json
Saved shit\rearrangement_00005\meta.json
Saved shit\rearrangement_00006\meta.json
Saved shit\rearrangement_00007\meta.json
Saved shit\rearrangement_00008\meta.json
Saved shit\rearrangement_00009\meta.json
Generating 10 scenes with 8 objects...
Saved shit\rearrangement_00010\meta.json
Saved shit\rearrangement_00011\meta.json
Saved shit\rearrangement_00012\meta.json
Saved shit\rearrangement_00013\meta.json
Saved shit\rearrangement_00014\meta.json
Saved shit\rearrangement_00015\meta.json
Saved shit\rearrangement_00016\meta.json
Saved shit\rearrangement_00017\meta.json
Saved shit\rearrangement_00018\meta.json
Saved shit\rearrangement_00019\meta.json
Generating 10 scenes with 9 objects...
Saved shit\rearrangement_00020\meta.json
Saved shit\rearrangeme

# Simulation

In [ ]:
from sim.utils import (
	Camera, PyBulletSim, load_object, load_table, get_size
)

def map_position(pos, x_range, y_range, grid_scale=0.01):
    x, y = pos

    # Map x
    new_x_min, new_x_max = x_range
    new_x = x * grid_scale * (new_x_max - new_x_min) + new_x_min

    # Map y
    new_y_min, new_y_max = y_range
    new_y = y * grid_scale * (new_y_max - new_y_min) + new_y_min

    return new_x, new_y

def get_available_body_types(objects):
    available = {}
    for obj in objects:
        model_name = obj['model_name']
        if model_name not in available:
            available[model_name] = len(os.listdir(f"objects/{model_name}"))
    return available

def choose_least_used_body_type(model_name, available_body_types, counter):
    count = counter[model_name]
    num_types = available_body_types[model_name]
    min_used = min(count.get(bt, 0) for bt in range(1, num_types + 1))
    candidates = [bt for bt in range(1, num_types + 1) if count.get(bt, 0) == min_used]
    chosen = random.choice(candidates)
    counter[model_name][chosen] += 1
    return chosen

def compute_position_and_orientation(obj, objects, base_z):
	stack_hierarchy = 0
	j = obj['base_id']
	while j is not None:
		stack_hierarchy += 1
		j = objects[j]['base_id']

	pos = [obj['pos'][0], obj['pos'][1], base_z + 0.05 + 0.085 * stack_hierarchy]
	orn = [0, 0, 0]

	if stack_hierarchy > 0 and obj['label'] in [0, 1, 2]:
		orn[0] = np.pi / 2
		j = obj['base_id']
		if objects[j]['label'] in [6, 7]:
			pos, orn = random_tilt(pos, orn, max_shift=0.01, tilt_angle=10)
		elif objects[j]['label'] == 8:
			pos, orn = random_tilt(pos, orn, max_shift=0.02, shift_end=True, tilt_angle=10)
		else:
			orn[2] = random.uniform(0, 2 * np.pi)
	else:
		orn[2] = random.uniform(0, 2 * np.pi)

	return pos, orn

def get_objects_from_scene_meta(scene_meta, mode, x_range, y_range, z, fixed_body_types=None):
	objects = []
	for i, obj in enumerate(scene_meta['objects']):
		label = int(obj['label'])
		model_name = OBJECTS[label]['name']
		position = obj['initial_pos'] if mode == "initial" else obj['target_pos']
		base_id = obj['initial_base_id'] if mode == "initial" else obj['target_base_id']

		objects.append({
			'object_id': i,
			'label': label,
			'model_name': model_name,
			'pos': map_position(position, x_range, y_range),
			'base_id': base_id,
		})

	available_body_types = get_available_body_types(objects)
	body_type_counter = defaultdict(Counter)

	for obj in objects:
		model_name = obj['model_name']
		pos, orn = compute_position_and_orientation(obj, objects, z)

		if fixed_body_types:
			chosen_type = fixed_body_types[obj['object_id']]
		else:
			chosen_type = choose_least_used_body_type(model_name, available_body_types, body_type_counter)

		body_id, _ = load_object(model_name, pos, orn, bodyType=chosen_type)
		obj['body_id'] = body_id
		obj['body_type'] = chosen_type

	return objects

# # sim = PyBulletSim(p.GUI)
# sim = PyBulletSim(p.DIRECT)
# table = load_table((100, 100))
# z = get_size(table)[2]
# x_range, y_range = [-0.5, 0.5], [-0.5, 0.5]
# cam = Camera(target_pos=[0.6, 0, 1.3], distance=0.2, yaw=90, pitch=-50, roll=0)

# ...
# objects = get_objects_from_scene_meta(scene_meta, "initial", x_range, y_range, z)

# sim.step(5)

# bboxes, labels = [], []
# for obj in objects:
# 	bboxes.append(cam.compute_bounding_box(obj['model_name'], obj['body_id'], obj['body_type']))
# 	labels.append(f"{obj['model_name']}_{obj['body_id']}")

# cam.draw_bounding_boxes(cam.capture_image(), bboxes, labels)
# # cam.show_img(cam.capture_image())

# sim.close()

In [3]:
def generate_image_and_label(scene_meta, folder_path, mode):
	assert mode in ('initial', 'target')

	image_path = os.path.join(folder_path, f'{mode}_image.png')
	label_path = os.path.join(folder_path, f'{mode}_labels.json')

	sim = PyBulletSim(p.DIRECT)
	table = load_table((100, 100))
	z = get_size(table)[2]

	x_range, y_range = [-0.5, 0.5], [-0.5, 0.5]
	cam = Camera(target_pos=[0.6, 0, 1.3], distance=0.2, yaw=90, pitch=-50, roll=0)

	fixed_body_types = None
	if mode == 'target':
		initial_labels_path = os.path.join(folder_path, 'initial_labels.json')
		with open(initial_labels_path, 'r') as f:
			initial_labels = json.load(f)
		fixed_body_types = {
			obj['obj_id']: int(obj['model_id'].split('_')[-1])
			for obj in initial_labels
		}

	objects = get_objects_from_scene_meta(scene_meta, mode, x_range, y_range, z, fixed_body_types)

	sim.step(5)

	json_data = []
	for obj in objects:
		bbox = cam.compute_bounding_box(obj['model_name'], obj['body_id'], obj['body_type'], numSamples=500)
		pos, orn = p.getBasePositionAndOrientation(obj['body_id'])
		euler = p.getEulerFromQuaternion(orn)
		json_data.append({
			"model_name": obj['model_name'],
			"model_id": f"{obj['model_name']}_{obj['body_type']}",
			"obj_id": int(obj['object_id']),
			"6D_pose": [float(x) for x in (pos + euler)],
			"natural_parent_list": [int(obj['base_id'])] if obj['base_id'] is not None else [],
			"bbox": [float(x) for x in bbox],
		})

	with open(label_path, 'w') as f:
		json.dump(json_data, f, indent=4)

	image = cam.capture_image()
	img = Image.fromarray(image)
	img.save(image_path)

	sim.close()

dataset_dir = 'dataset'
rearrangements = sorted([
    d for d in os.listdir(dataset_dir)
    if os.path.isdir(os.path.join(dataset_dir, d)) and d.startswith('rearrangement_')
])

print(f'Found {len(rearrangements)} rearrangement folders.')

pbar = tqdm(rearrangements, desc="Processing rearrangements")

for folder_name in pbar:
    folder_path = os.path.join(dataset_dir, folder_name)
    meta_path = os.path.join(folder_path, 'meta.json')

    if not os.path.exists(meta_path):
        pbar.set_description(f'Skipping {folder_name}: no meta')
        continue

    with open(meta_path, 'r') as f:
        scene_meta = json.load(f)

    initial_image_path = os.path.join(folder_path, 'initial_image.png')
    target_image_path = os.path.join(folder_path, 'target_image.png')

    if not os.path.exists(initial_image_path):
        pbar.set_description(f'{folder_name}: Generating initial')
        generate_image_and_label(scene_meta, folder_path, mode='initial')

    if not os.path.exists(target_image_path):
        pbar.set_description(f'{folder_name}: Generating target')
        generate_image_and_label(scene_meta, folder_path, mode='target')

    # Reset to default after each iteration
    pbar.set_description("Processing rearrangements")


Found 1200 rearrangement folders.


Processing rearrangements: 100%|██████████| 1200/1200 [4:13:07<00:00, 12.66s/it]                


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def draw_bboxes(ax, image_path, label_path, title):
    # Load image
    image = Image.open(image_path)
    ax.imshow(image)
    ax.set_title(title)
    ax.axis('off')

    # Load labels
    with open(label_path, 'r') as f:
        labels = json.load(f)

    # Draw each bounding box
    for obj in labels:
        bbox = obj['bbox']
        xmin, ymin, xmax, ymax = bbox
        width = xmax - xmin
        height = ymax - ymin

        rect = patches.Rectangle(
            (xmin, ymin), width, height,
            linewidth=2, edgecolor='yellow', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(xmin, ymin - 10, f"{obj['obj_id']}_{obj['model_name']}", color='yellow', fontsize=8)

def visualize_rearrangement(rearrangement_path):
    initial_img = os.path.join(rearrangement_path, 'initial_image.png')
    target_img = os.path.join(rearrangement_path, 'target_image.png')
    initial_lbl = os.path.join(rearrangement_path, 'initial_labels.json')
    target_lbl = os.path.join(rearrangement_path, 'target_labels.json')

    if not all(os.path.exists(p) for p in [initial_img, target_img, initial_lbl, target_lbl]):
        print("One or more files are missing.")
        return

    # Extract scene ID from folder name
    scene_id = os.path.basename(rearrangement_path).split('_')[-1]

    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    draw_bboxes(axes[0], initial_img, initial_lbl, title='Initial Scene')
    draw_bboxes(axes[1], target_img, target_lbl, title='Target Scene')

    fig.suptitle(f'Scene ID: {scene_id}')
    plt.tight_layout()
    plt.show()

dataset_dir = 'dataset'
folders = [d for d in os.listdir(dataset_dir) if d.startswith('rearrangement_')]
chosen = random.choice(folders)
visualize_rearrangement(os.path.join(dataset_dir, chosen))